In [37]:
import os
from glob import glob

import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt

### 함수 정의

In [ ]:
def for_tokenizer(tokenizer):
    threshold=3 # 이것들 구하는 이유를 확실히 하자 *** 빈도수 적은 건 빼는게 나음.

    total_cnt=len(tokenizer.word_index)
    rare_cnt=0 # threshold보다 작은 단어의 수  ?
    total_freq=0 # 전체 단어 빈도수의 합
    rare_freq=0 # 빈도수의 총 합.

    for key,value in tokenizer.word_counts.items(): #역시, 하나만 쓰여도 2개로 받아야 한다.
        total_freq+=value

        if(value<threshold):  #OrderedDict ? *** 
            rare_cnt+=1
            rare_freq+=value
    vocab_size=total_cnt-rare_cnt+2 # 0,1번의 패딩과 OOV토큰 고려
    print('단어 집합의 크기 :', vocab_size)
    return vocab_size
                                      

### 정제함수 , 완료 : 한국어 // 미완료 : 영어, 이모티콘
> 한글만 사용 가능  
* INPUT : 원본 댓글.csv의 DataFrame 형식  ( pd.read_csv('comment.csv') )  
* OUTPUT : 정제 후 댓글.csv의 DataFrame 형식 ( ~.to_csv('comment_refined.csv')로 저장)


In [ ]:
def refining (comment_csv): # 입력으로 댓글 csv하나를 받는다. (pd.Dataframe 형식으로 )
    prev_len=len(comment_csv) # 정제 전 댓글 개수
    
    #한글 외 댓글들은 '    '처럼 모두 공백 or null값으로 저장. 
    comment_csv['comment_refined']=comment_csv['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #띄어쓰기까지
    #공백댓글 인덱스 추출
    blank_index=[index for index,comment in enumerate(comment_csv['comment_refined']) if (set(comment)=={' '})]
    #공백댓글 제거
    comment_csv.drop(blank_index,inplace=True)
    #null댓글 제거
    comment_csv['comment_refined'].replace('', np.nan, inplace=True) #empty comment를 null로.
    comment_csv.dropna(how='any',inplace=True) # null값 삭제
    #인덱스 초기화
    comment_csv.reset_index(drop=True,inplace=True)
    after_len=len(comment_csv)
    
    #열 순서 변경
    comment_csv=comment_csv[['author','comment','comment_refined','like']]
    print(f'[{prev_len}개 -->  {after_len}개]')
    
    #정제 문자열이 추가된 Dataframe 반환. 
    return comment_csv

# 보강 : 물음표, 특수문자, 이모티콘, 영어
# 정제 된 댓글은 konlpy의 형태소 분석기를 이용하여 시뮬레이션을 해본다.
# 정제 후 저장되는 폴더만 다를 뿐, 댓글 csv의 이름은 동일하다.


#INPUT
# folder : 주피터 노트북 파일과 같은 경로에 있는 폴더 이름
# meta_path_list : 위에서 정의한 (BJ)_meta.csv의 경로 array
# channel_path_list : 위에서 정의한 folder/(BJ) 의 경로 array 
# channel_list : BJ list.
# opt : 용량이 크기 때문에, 잘못 저장되면 큰일나므로 기본적으로 상위 1개의 폴더에 대해서만 진행
# opt = 'test' : BJ 1개의 폴더만 저장 진행 /// opt = 'full' : BJ 전체의 폴더에 대해 저장 진행
def all_refining(folder,meta_path_list,channel_path_list,channel_list): 
    #새로 저장할 폴더 만들기 (폴더 이름 : comment_refined)
    for bj in os.listdir(folder):
        try:
            os.mkdir(os.path.join(folder,bj,'comments_refined'))
        except:
            pass
        
    before_num=[]
    after_num=[]
    # meta_path_list = [././오스틴_meta.csv,     ./.버들Buddle_meta.csv , .....]
    for index, meta_path in enumerate(meta_path_list): 
        meta_temp=pd.read_csv(meta_path) # 1개의 meta파일 불러오기.
        
        #channel 하나당, 총 댓글 meta file 생성
        meta_comment=pd.DataFrame(columns=['channel','title','author','comment','comment_refined','like'])
        
        
        num=len(meta_temp)-1 #동영상 개수
        before_num.append(num) # 변경 이전 동영상 개수 지정
        error_count=0 #동영상을 불러오지 못한 개수 카운팅
        for idx,(channel,title) in enumerate(meta_temp[['channel','title']].to_numpy()):
            
            try: #하나의 댓글 불러오기, 가끔 인코딩 오류 뜰 수도 있음 #오류 확인해야함 ***
                temp=pd.read_csv(os.path.join(channel_path_list[index],r'comments\{}.csv'.format(title)),engine='python',encoding='UTF-8')

            except :
                error_count+=1
                print(f'댓글 로드 실패 : [{idx}/{num}] [{channel}] <{title}>')
                continue
                
            try: #간혹, 정제할 때 오류 뜰 수도 있음
                temp=refining(temp) # 우선, 한국어만 변환, title도 저장
            except:
                error_count+=1
                print(f'댓글 정제 실패 : [{idx}/{num}] [{channel}] <{title}>')
                continue
            # 저장
            #열 추가 생성
            temp['channel']=channel
            temp['title']=title
            temp=temp[['channel','title','author','comment','comment_refined','like']] #순서조정
            meta_comment=meta_comment.append(temp)
            print(f'댓글 변환 성공 : [{idx}/{num}] [{channel}] <{title}>') 
        #################이거 to_csv대신 to_pickle?
        meta_comment.to_csv(os.path.join(channel_path_list[index],r'{}_meta_comment.csv'.format(channel_list[index])),index=False)
        after_num.append(num-error_count)
        
    print('------------------------------------------------------------------')
    print('실행 종료')
    for i in range(len(before_num)):
        print('[{}]채널의 댓글 정제 결과 , 변경 전 동영상 개수: {} 변경 후 동영상 개수 : {}'.format(channel_list[i],before_num[i],after_num[i]))
        
    
    
    
    

### 댓글 예측 함수(개별모니터링용, 채널전체용)

In [ ]:
# 단순 모니터링용, meta_comment에 영향 xs
# 위에서 tokenizer를 가져와야 쓸 수 있다.
def sentiment_predict(meta_comment,start,end): #이는 모니터링 용으로, [start:end]로 slicing함.
    meta_comment=pd.read_csv(meta_comment) # meta_comment : 의 경로
    X_train=[]
    okt=Okt() #형태소 분석기
    original=meta_comment['comment'][start:end]
    original=original.to_frame()
    original.columns=['comment']
    score_array=[]
    for index,sentence in enumerate(meta_comment['comment_refined'][start:end]): # 토큰화
        temp_X=okt.morphs(sentence,stem=True)
        #이거 lstm에서는 하나 빼야함
        temp_X=[word for word in temp_X if not word in stopwords_movie]
        #temp_X=[word for word in temp_X if not word in stopwords_shop]
        encoded=tokenizer2.texts_to_sequences([temp_X])
        pad_new=pad_sequences(encoded,maxlen=30)        
        score=float(loaded_model2.predict(pad_new))
        print(original.values[index])
        score_array.append(score*100)
        if(score > 0.5):
            print("--> ","{:.2f}% 확률로 긍정적인 댓글입니다.\n".format(score * 100))
            
        else:
            print("-->","{:.2f}% 확률로 부정적인 댓글입니다.\n".format((1 - score) * 100))
            
    original["sentiment_score"]=score_array
    return original

# 단순 모니터링용, meta_comment에 영향 xs
# 위에서 tokenizer를 가져와야 쓸 수 있다.
# meta_comment : 경로. 
# output : 전체 결과의 meta_comment_predict1(1,2,3... ) 생성(모델마다 비교해야함)
def sentiment_predict_all(meta_comment):
    meta_comment=pd.read_csv(meta_comment) # meta_comment : 의 경로
    X_train=[]
    okt=Okt() #형태소 분석기
    original=meta_comment['comment']
    for sentence in meta_comment['comment_refined'][start:end]: # 토큰화
        temp_X=okt.morphs(sentence,stem=True)
        temp_X=[word for word in temp_X if not word in stopwords_movie]
        temp_X=[word for word in temp_X if not word in stopwords_shop]
        X_train.append(temp_X)
    encoded=tokenizer.texts_to_sequences(X_train)
    print(encoded)
    pad_new=pad_sequences(encoded,maxlen=max_len)
    score_array=[]
    for index,comment in enumerate(pad_new):
        score=float(loaded_model.predict(comment))
        if(score > 0.5):
            print(comment)
            print("--> ","{:.2f}% 확률로 긍정적인 댓글입니다.\n".format(score * 100))
            socre_array.append(score*100)
        else:
            print(comment)
            print("-->","{:.2f}% 확률로 부정적인 댓글입니다.\n".format((1 - score) * 100))
            score_array.append(score*100)
    original["sentiment_score"]=score_array
    return original

## 폴더 내 댓글들 정제  

                                            

In [287]:
folder='크롤링폴더' # 크롤링 할 폴더 이름 (주피터노트북과 같은 경로에 있을 때)

In [288]:
# 폴더 내 모든 정보 수집
channel_list=os.listdir(folder) #채널 이름 list
channel_path_list=glob(r'{}\*'.format(folder)) #채널 경로 list (위랑 같아야 함, 다르다면 수동 설정)
meta_path_list=[]
for i in range(len(channel_list)):
    meta_path_list.append(os.path.join(channel_path_list[i],
                                       '{}_meta.csv'.format(channel_list[i])))
    

In [289]:
print('크롤링 할 채널들 : ',channel_list)
print('크롤링 할 채널 경로들 : ',channel_path_list)
print('메타 파일 경로들 : ', meta_path_list)


크롤링 할 채널들 :  ['BENI 유베니', 'Eating what is Given 오스틴', '가오니의 메뉴판GAONI', '리폴LeePaul', '밍칭mingching', '버들Buddle', '부엉이TV', '살빼조DietJo', '세리티 Serity', '유노-U_know', '캐피 KEPHI']
크롤링 할 채널 경로들 :  ['크롤링폴더\\BENI 유베니', '크롤링폴더\\Eating what is Given 오스틴', '크롤링폴더\\가오니의 메뉴판GAONI', '크롤링폴더\\리폴LeePaul', '크롤링폴더\\밍칭mingching', '크롤링폴더\\버들Buddle', '크롤링폴더\\부엉이TV', '크롤링폴더\\살빼조DietJo', '크롤링폴더\\세리티 Serity', '크롤링폴더\\유노-U_know', '크롤링폴더\\캐피 KEPHI']
메타 파일 경로들 :  ['크롤링폴더\\BENI 유베니\\BENI 유베니_meta.csv', '크롤링폴더\\Eating what is Given 오스틴\\Eating what is Given 오스틴_meta.csv', '크롤링폴더\\가오니의 메뉴판GAONI\\가오니의 메뉴판GAONI_meta.csv', '크롤링폴더\\리폴LeePaul\\리폴LeePaul_meta.csv', '크롤링폴더\\밍칭mingching\\밍칭mingching_meta.csv', '크롤링폴더\\버들Buddle\\버들Buddle_meta.csv', '크롤링폴더\\부엉이TV\\부엉이TV_meta.csv', '크롤링폴더\\살빼조DietJo\\살빼조DietJo_meta.csv', '크롤링폴더\\세리티 Serity\\세리티 Serity_meta.csv', '크롤링폴더\\유노-U_know\\유노-U_know_meta.csv', '크롤링폴더\\캐피 KEPHI\\캐피 KEPHI_meta.csv']


### 폴더 내 comments 정제 - 한국어 , 영어,이모티콘(미완)
> 실행 결과 ~_meta_comment.csv가 생김

In [9]:
pd.read_csv('버들Buddle_meta.csv')

,channel,subscribe,title,view,duration,date,description,comment,like,dislike,thumbnail
0,버들Buddle,90000,[와차밥] 갓담은 김치 하나면 됩니다 먹방 요리 레시피 🌶🌶 Korean Kimch...,39283,9.70,2020-11-02,✅Turn on subtitle settings.\n\n구독 좋아요는 사랑입니다❤\...,161,1820,57,https://i.ytimg.com/vi/9A_pTN6fHeg/maxresdefau...
1,버들Buddle,90000,[와차밥]햄 가득 부대찌개 라면사리 치즈닭갈비 먹방 Budae jjigae(Spic...,31884,12.65,2020-10-31,✅Turn on subtitle settings.\n\n구독 좋아요는 사랑입니다❤ ...,79,1349,33,https://i.ytimg.com/vi/i1MmA1svaO0/maxresdefau...
2,버들Buddle,90000,[와차밥] 신라면 대파김치 콘치즈밥전 먹방 Spicy Noodles & Cheese...,52445,10.43,2020-10-29,✅Turn on subtitle settings.\n\n구독 좋아요는 사랑입니다❤ ...,94,1683,51,https://i.ytimg.com/vi/sQe9-EzszC8/maxresdefau...
3,버들Buddle,90000,[와차밥] LA갈비 부추무침 먹방 요리 레시피 LA Galbi(Beef rib) M...,61211,13.10,2020-10-27,✅Turn on subtitle settings.\n\n구독 좋아요는 사랑입니다❤ ...,109,2081,59,https://i.ytimg.com/vi/oEMKvwETpTI/maxresdefau...
4,버들Buddle,90000,[와차밥] 육회비빔밥 꽃게된장찌개 먹방 요리 Korean Home Made Food...,27966,10.65,2020-10-25,✅Turn on subtitle settings.\n\n구독 좋아요는 사랑입니다❤\...,75,1273,33,https://i.ytimg.com/vi/TnxNVpH9KeM/maxresdefau...
...,...,...,...,...,...,...,...,...,...,...,...
310,버들Buddle,90000,BHC 뿌링클 닭다리 치즈볼 먹방 Chicken Cheese ball REAL SO...,3283,19.42,2019-02-01,#BHC뿌링클#치즈볼#먹방#mukbang\n좋아요와 구독 눌러주세요 ! ❤️\n시청...,23,125,3,https://i.ytimg.com/vi/-3dKTUprGgw/maxresdefau...
311,버들Buddle,90000,문현전통 한우 곱창전골 먹방 REAL SOUND ASMR MUKBANG EATING...,5078,14.30,2019-01-30,#곱창전골#먹방#mukbang\n구독과 좋아요 눌러주세요 ! ❤️\n시청해주셔서 감...,29,137,9,https://i.ytimg.com/vi/dblCYv4rh2g/maxresdefau...
312,버들Buddle,90000,짜파게티 3봉 통단무지 먹방 Black been noodles REAL SOUND ...,2698,12.37,2019-01-29,#짜파게티#통단무지#먹방#mukbang\n구독과 좋아요 눌러주세요 ! ❤️\n시청해...,20,115,3,https://i.ytimg.com/vi/T9QIzXPad-s/maxresdefau...
313,버들Buddle,90000,통삼겹 고추냉이 비빔국수 먹방 Pork belly Spicy noodle REAL ...,2284,13.63,2019-01-27,#통삼겹#비빔국수#먹방#mukbang\n구독과 좋아요 눌러주세요! ❤️\n시청해주셔...,15,110,2,https://i.ytimg.com/vi/YWGknEqWHgs/maxresdefau...


In [292]:
all_refining(folder,meta_path_list,channel_path_list,channel_list)

[106개 -->  103개]
댓글 변환 성공 : [0/62] [유노-U_know] <한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶음면 2개 먹고 4분 44초 버티기! korean eating show K food fighter>
댓글 로드 실패 : [1/62] [유노-U_know] <빵이 16종이나!? 미슐랭 별 3개 빵집 다 털어 왔습니다!! 빵 16종 먹방!! korean eating show korean food fighter>
댓글 로드 실패 : [2/62] [유노-U_know] <불마왕 라면 2개를 12초컷..? 아니;; 라면이 있었는데.. 없어요;; ㅋㅋㅋㅋㅋㅋㅋㅋ 최군님 푸파대회에서 1등하고 왔습니다! korean food fighter>
[129개 -->  124개]
댓글 변환 성공 : [3/62] [유노-U_know] <실비김치 광고 받았는데 너무 솔직하게 리뷰해서 죄송합니다 광고주님.. 실비김치에 짜장면과 떡만둣국 먹방!! korean eating show korean food fighter>
[65개 -->  59개]
댓글 변환 성공 : [4/62] [유노-U_know] <마피아 떡볶이 고기 떡볶이에 튀김 8종 먹방!! 저는 이 떡볶이 원픽입니다.. Korean eating show korean eating challenge>
[76개 -->  70개]
댓글 변환 성공 : [5/62] [유노-U_know] <단짠단짠 최고!! 밥 한 대접에 국물 치즈두루치기 먹방! korean eating show korean food fighter>
댓글 로드 실패 : [6/62] [유노-U_know] <협찬 받았는데..대왕떡볶이 도전먹방..? 20분 안에 다 먹으면 트로피에 10만원까지 주신대요! 맛난맛떡볶이  korean eating show korean food fighter>
[52개 -->  46개]
댓글 변환 성공 : [7/62] [유노-U_know] <고기국수 6인분에 삼겹살1kg!! 완전 꿀조합입니다..

[53개 -->  30개]
댓글 변환 성공 : [5/82] [캐피 KEPHI] <삼첩분식 감자폭탄 치즈떡볶이 곱창구이 모듬튀김 먹방! CHEESE TTEOKBOKKI Eating Sounds MUKBANG>
댓글 로드 실패 : [6/82] [캐피 KEPHI] <커다란 피자 세 조각 먹방(feat.버팔로윙) / GIANT PIZZA & CHICKEN WINGS MUKBANG>
[92개 -->  60개]
댓글 변환 성공 : [7/82] [캐피 KEPHI] <돼지고기 김치찌개와 스팸 계란말이 집밥먹방 KOREAN HOME FOOD Eating Sounds MUKBANG>
[124개 -->  66개]
댓글 변환 성공 : [8/82] [캐피 KEPHI] <베이컨 까르보나라 파스타 그리고 치즈 돈까스 먹방 BACON CARBONARA PASTA & CHEESE PORK CUTLET eating sounds MUKBANG>
[80개 -->  54개]
댓글 변환 성공 : [9/82] [캐피 KEPHI] <매콤한 알찜에 곤이전 날치알주먹밥 먹방! 마무리는 계란에 김넣고 비빔밥 BRAISED SEAFOOD & SEAFOOD PANCAKE Eating Sounds MUKBANG>
[75개 -->  39개]
댓글 변환 성공 : [10/82] [캐피 KEPHI] <여러가지 생크림 와플과 꽈배기 먹방 WAFFLE & GGWABAEGI(Twisted Bread Stick) Eating Sounds MUKBANG>
[51개 -->  23개]
댓글 변환 성공 : [11/82] [캐피 KEPHI] <GS25 X 편의점 샛별이 신메뉴(도시락, 김밥, 과일, 마카롱) 편의점먹방 KOREAN CONVENIENCE STORE FOOD Eating Sounds MUKBANG>
[76개 -->  43개]
댓글 변환 성공 : [12/82] [캐피 KEPHI] <앵그리짜파구리 뿌링클 사이드메뉴(치즈볼, 핫도그, 감자튀김) 먹방 feat.통단무지 CHAPAGURI(Ram-don) & BBURINKLE Eat

In [291]:
# 폴더 내 모든 정보 수집
folder='크롤링폴더2'
channel_list=os.listdir(folder) #채널 이름 list
channel_path_list=glob(r'{}\*'.format(folder)) #채널 경로 list (위랑 같아야 함, 다르다면 수동 설정)
meta_path_list=[]
for i in range(len(channel_list)):
    meta_path_list.append(os.path.join(channel_path_list[i],
                                       '{}_meta.csv'.format(channel_list[i])))
    

In [13]:
pd.read_csv(os.path.join(channel_path_list[2],r'{}_meta_comment.csv'.format(channel_list[2])),)

,channel,title,author,comment,comment_refined,like
0,부엉이TV,오늘날씨 완전겨울 보글보글 된장찌개와 코다리조림 입천장 까져도 즐거운맛,신준혁,오늘도 맛있는 영상 잘봤습니다 영상과는 상관없는 댓글 올리는점 죄송하다는 말씀드리구...,오늘도 맛있는 영상 잘봤습니다 영상과는 상관없는 댓글 올리는점 죄송하다는 말씀드리구...,12
1,부엉이TV,오늘날씨 완전겨울 보글보글 된장찌개와 코다리조림 입천장 까져도 즐거운맛,Hotel California,보글보글 된장찌게 끓는소리가 온몸이 따듯해 지는듯 낼은 더 춥다네요~^^\n넘 맛있...,보글보글 된장찌게 끓는소리가 온몸이 따듯해 지는듯 낼은 더 춥다네요넘 맛있게 드셔주...,7
2,부엉이TV,오늘날씨 완전겨울 보글보글 된장찌개와 코다리조림 입천장 까져도 즐거운맛,박경옥,개인적으로 코다리 조림 무쟈게 좋아라 합니다. 음식두 잘하시나봐요 오늘도 맛나게 차...,개인적으로 코다리 조림 무쟈게 좋아라 합니다 음식두 잘하시나봐요 오늘도 맛나게 차분...,5
3,부엉이TV,오늘날씨 완전겨울 보글보글 된장찌개와 코다리조림 입천장 까져도 즐거운맛,김혜민,역시~~한국사람은된장찌게에흰쌀밥이죠~~😊😚,역시한국사람은된장찌게에흰쌀밥이죠,5
4,부엉이TV,오늘날씨 완전겨울 보글보글 된장찌개와 코다리조림 입천장 까져도 즐거운맛,ᄋᄋ,오늘같이 추운 날씨에는 역시 따뜻한 국물을 먹어야 힘이나죠!!,오늘같이 추운 날씨에는 역시 따뜻한 국물을 먹어야 힘이나죠,7
...,...,...,...,...,...,...
9629,부엉이TV,제주에서 제일 잘 나가는 어묵집먹방.mukbang.eating show.,김빵이모1003,보기 좋아요~,보기 좋아요,1
9630,부엉이TV,제주에서 제일 잘 나가는 어묵집먹방.mukbang.eating show.,서광일,둘째 가라면 서러워 해도 됩니다 진국은 아래쪽 200와 400지점에 있는곳이 1.2...,둘째 가라면 서러워 해도 됩니다 진국은 아래쪽 와 지점에 있는곳이 등 입니다 서부는...,0
9631,부엉이TV,제주에서 제일 잘 나가는 어묵집먹방.mukbang.eating show.,ᅳᅳ,옆에서 아드님이 재미를 더해주는데 왜 자꾸 말리세요~ 영상은 찍어야 하는데 ㅋㅋ 아...,옆에서 아드님이 재미를 더해주는데 왜 자꾸 말리세요 영상은 찍어야 하는데 ㅋㅋ 아들...,2
9632,부엉이TV,제주에서 제일 잘 나가는 어묵집먹방.mukbang.eating show.,찌니,잘생긴 아드님 먹방이 훈훈 한데 ~~말리시네요,잘생긴 아드님 먹방이 훈훈 한데 말리시네요,1


### 댓글 예측 함수(개별모니터링용, 채널전체용)

In [201]:
from tensorflow.keras.models import load_model
loaded_model=load_model('best_model2.h5')
loaded_model2=load_model('best_model.h5')
#모델마다
# tokenizer, max_len 저장해야함 (30,65) (두개 나눠서 tokenizer 불러오는 함수를 만들자.)

In [371]:
loaded_model2=load_model('best_model.h5')

In [206]:
loaded_model.predict

<bound method Model.predict of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000023487CEF4C8>>

In [207]:
loaded_model.predict([[1,2,3,20]])

array([[0.21471]], dtype=float32)

In [23]:
#폴더 내 모든 출력을 위해서.
meta_comment_path_list=[]
for i in range(len(channel_list)):
    meta_comment_path_list.append(os.path.join(channel_path_list[i],
                                       '{}_meta_comment.csv'.format(channel_list[i])))

In [38]:
# 불용어 : 굉장히 중요하다. *** --> 데이터를 살핌으로써 진행할 수 있다. 
stopwords_shop=['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
stopwords_movie=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords=stopwords_movie

In [ ]:
# tokenizer를 불러올 수는 없나 ? 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [130]:
# tokenizer fitting.
total_data=pd.read_csv('total_tokenized(0.4k).csv',converters={'document': eval})
X_train=total_data['document'].values
y_train=total_data['label'].values
tokenizer=Tokenizer() #keras의 전처리기
tokenizer.fit_on_texts(X_train)


In [364]:
X_train2=pd.read_csv('movie_tokenized(0.15k).csv',converters={'document': eval},header=None)

In [ ]:
X_train2

In [161]:

X_train2=pd.read_csv('movie_tokenized(0.15k).csv',converters={'document': eval},header=None)
X_train2=X_train2[1].values
tokenizer2=Tokenizer() #keras의 전처리기
tokenizer2.fit_on_texts(X_train2)


In [ ]:
print(tokenizer.word_index)

In [132]:
# 토큰화용, 빈도수가 적은 단어 제외.
vocab_size=for_tokenizer(tokenizer)
tokenizer=Tokenizer(vocab_size, oov_token='OOV')
tokeniOzer.fit_on_texts(X_train)

단어 집합의 크기 : 30706
30706


In [162]:
vocab_size2=for_tokenizer(tokenizer2)
tokenizer2=Tokenizer(vocab_size2, oov_token='OOV')
tokenizer2.fit_on_texts(X_train2)

단어 집합의 크기 : 19417


In [59]:
def for_tokenizer(tokenizer):
    threshold=3 # 이것들 구하는 이유를 확실히 하자 *** 빈도수 적은 건 빼는게 나음.

    total_cnt=len(tokenizer.word_index)
    rare_cnt=0 # threshold보다 작은 단어의 수  ?
    total_freq=0 # 전체 단어 빈도수의 합
    rare_freq=0 # 빈도수의 총 합.

    for key,value in tokenizer.word_counts.items(): #역시, 하나만 쓰여도 2개로 받아야 한다.
        total_freq+=value

        if(value<threshold):  #OrderedDict ? *** 
            rare_cnt+=1
            rare_freq+=value
    vocab_size=total_cnt-rare_cnt+2 # 0,1번의 패딩과 OOV토큰 고려
    print('단어 집합의 크기 :', vocab_size)
    return vocab_size
                                      

In [142]:
# 한 번만 실행하면 된다. 
max_len=65
# 불용어 : 굉장히 중요하다. *** --> 데이터를 살핌으로써 진행할 수 있다. 
stopwords_shop=['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
stopwords_movie=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


In [ ]:
start=30
end=40
meta_comment=test_path
meta_comment=pd.read_csv(meta_comment) # meta_comment : 의 경로
X_train3=[]
okt=Okt() #형태소 분석기
original=meta_comment['comment'][start:end]
original
ro
for sentence in meta_comment['comment_refined'][start:end]: # 토큰화
    temp_X=okt.morphs(sentence,stem=True)
    temp_X=[word for word in temp_X if not word in stopwords_movie]
    temp_X=[word for word in temp_X if not word in stopwords_shop]
    X_train3.append(temp_X)
print(X_train3)
encoded3=tokenizer.texts_to_sequences(X_train)
print(encoded3)
pad_new=pad_sequences(encoded3,maxlen=max_len)
print(pad_new)
score_array=[]
for index,comment in enumerate(pad_new):
    score=float(loaded_model.predict(comment))
    if(score > 0.5):
        print(comment)
        print("--> ","{:.2f}% 확률로 긍정적인 댓글입니다.\n".format(score * 100))
        socre_array.append(score*100)
    else:
        print(comment)
        print("-->","{:.2f}% 확률로 부정적인 댓글입니다.\n".format((1 - score) * 100))
        score_array.append(score*100)
original["sentiment_score"]=score_array

In [323]:
test_path='유노-U_know_meta_comment.csv'

In [351]:
sol

,comment,GRU,LSTM
30,진짜 매운가보다 ㅠㅠ,58.147854,36.446673
31,방금푸파보고 왔습니다 레게년,37.591982,35.978538
32,ㅋㅋㅋㅋㅋ야이씨 이건 경품줘라 진짜 찐텐반응 ㄷ ㄷ....보면서 재밌긴한데 정말 고...,44.564611,49.602595
33,원칩 먹으면 큰일나겠다ㅋㅋㅋㅋㅋㅋ,5.131057,36.788368
34,에고 내 속이 아파라 ㅠㅠ 수고하셨어용 ㅠㅠ,10.676727,36.788368
35,나중에 꼭 세계대회 챔피언 되셧으면 좋겟다요 ㅜㅜ,90.991348,38.801748
36,아이구 \n이번 한번으로 이제 이런고통 스러운건 하지 않기로ㅜ\n성공 축하ㅜㅜ,66.523647,41.538233
37,손가락....개쩔어...,82.659209,36.446673
38,ㅜㅜ 늘 열심히 하는거 같아서 좋은데 건강에 무리만 안갔으면 하는 마음 ㅜ....다...,95.332766,50.369990
39,맴 아퍼 죽것네 ㅠ,70.766234,35.581738


In [357]:
sol 

,comment,LSTM,GRU
800,아니.. 양을 거의 두배로 늘리고 시간을 줄이는건.. 이건 도전메뉴가 아니고.. 유...,49.754626,0.564322
801,많이 먹는거 보다 맛있게 믝는 컨셉이 더 좋다 목소리도 좋고 말도 잘해서,40.706700,92.167813
802,흥하시길!,40.724164,86.054492
803,그냥 조용히 먹는 모습도 좋은데...... 너무,35.978538,84.207612
804,노력하는 남자 유노 멋진남자 유노 힘내라~~힘 무조건 전진\n~~~~~,39.794469,92.630523
805,🏆🥇최고예요 유노~👋🙆‍♀️,36.446673,98.083729
806,진짜 잘먹는다 대리만족을,35.581738,96.878970
807,아프리카tv 어제 마피아 떡볶이 보고 왔어요 구독 앞으로 홧팅입니다\n햄식이 ...,50.208956,10.833231
808,앗ㅡ일빠넹ㅡ응원해요 유노씨,36.788368,89.370263
809,왤케 많이 먹어 윤호,35.581738,38.357618


In [356]:
val.columns=['comment','LSTM','GRU']

In [355]:
val['ve2']=val2['sentiment_score'].values

In [354]:
val2=sentiment_predict(test_path,800,900)

['아니.. 양을 거의 두배로 늘리고 시간을 줄이는건.. 이건 도전메뉴가 아니고.. 유튜버들 놀리는거 같아서 화나네요..상해기님꺼 유노님꺼 히밥님꺼 영상 다 봤는데.. 솔직히 유노님 상해기님 못드실 정도면 일반인은 도전 엄두도 못낼 양..']
--> 99.44% 확률로 부정적인 댓글입니다.

['많이  먹는거 보다  맛있게  믝는 컨셉이 더 좋다 목소리도 좋고 말도 잘해서']
-->  92.17% 확률로 긍정적인 댓글입니다.

['흥하시길!']
-->  86.05% 확률로 긍정적인 댓글입니다.

['그냥 조용히 먹는 모습도 좋은데...... 너무']
-->  84.21% 확률로 긍정적인 댓글입니다.

['노력하는 남자  유노  멋진남자 유노  힘내라~~힘 무조건  전진\n~~~~~']
-->  92.63% 확률로 긍정적인 댓글입니다.

['🏆🥇최고예요  유노~👋🙆\u200d♀️']
-->  98.08% 확률로 긍정적인 댓글입니다.

['진짜 잘먹는다 대리만족을']
-->  96.88% 확률로 긍정적인 댓글입니다.

['아프리카tv 어제  마피아 떡볶이 보고  왔어요 구독  앞으로 홧팅입니다\n햄식이 같은  방송하시면 절대 안됨 갠방 전혀 안하고 모기짖만 하면 절대안됨']
--> 89.17% 확률로 부정적인 댓글입니다.

['앗ㅡ일빠넹ㅡ응원해요 유노씨']
-->  89.37% 확률로 긍정적인 댓글입니다.

['왤케 많이 먹어 윤호']
--> 61.64% 확률로 부정적인 댓글입니다.

['처음 보는데 목소리가 너무 좋아요']
-->  98.05% 확률로 긍정적인 댓글입니다.

['새벽에 이 영상을보다니.... 떡볶이가 너무 먹고 싶어졌다....']
-->  92.39% 확률로 긍정적인 댓글입니다.

['아구 맛 표현 명랑하네요^^ 꿀성대 꿀조합']
-->  90.43% 확률로 긍정적인 댓글입니다.

['맛깔나게먹는다진짜']
-->  83.63% 확률로 긍정적인 댓글입니다.

['그동안 돈내고 못먹은거 유노님이 대신 먹었다고 생각할게요~ 돈 안아까워~ ㅎㅎ']
--

['안녕하세요 여러분! 유노입니당!\n오늘은 바로바로 무한리필의 원조격인!\n엉터리 생고기 무한리필에 다녀왔어요!\n여기는 서울 강서구 발산역 쪽에 위치한\n엉터리생고기 마곡사이언스점인데요!\n미리 사전에 제가 좀 많이 먹는데\n계속 계속 리필 받아먹어도 괜찮을까요라고\n여쭤봤는데 얼마든지 먹어도 된다고 하셔서\n이번에 삼겹살 약 4키로를.. 털고 왔어요\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n죄송합니다.. 사장님...\n아 근데도 마지막에 결제하고 나가려고 하니까\n사장님께서 공짜로 주실려고 하셔서\n진짜 감동먹었었는데!\n음식값은 그래도 다행히 결제하고 나왔어요!\n진짜 오랜만에 고기 폭식하니까\n이 말이 진짜 와닿더라고요..\n기분이 저기압일땐 고기앞으로 가라...\n진짜 띵언...\n여러분도 오늘 하루? 내일?\n맛있는 지글지글 익은 삼겹살\n어떠신가요!?\n저는 이만! 물러나구요!\n영상 재밌게 시청해주시고!\n앞으로도 더 맛있는 영상으로\n찾아뵐게요! 안뇽!!!\n오늘 하루도 수고많으셨어요!!']
-->  92.86% 확률로 긍정적인 댓글입니다.

['울산의 청년 윤호군~\n울산에서 응원합니다~\n홧팅♡♡♡']
-->  92.50% 확률로 긍정적인 댓글입니다.

['이 분 진짜 맛있게 많이 깔끔하게 잘 드시는데 ㅠㅠ 얼른 더더욱 뜨셨으면 좋겠습니다!유노님 화이팅~!!! -구독자 3천명때부터 보던 독자 올림.']
-->  99.38% 확률로 긍정적인 댓글입니다.

['잘 봤습니다. 근데 주실 수 있을까요? 라는 말보다 주세요가 맞는 표현 일듯. 더더욱 번창하세요']
-->  99.31% 확률로 긍정적인 댓글입니다.

['유노님도  얼렁 떡상해야델텐데']
-->  56.68% 확률로 긍정적인 댓글입니다.

['착하고 겸손한 왕자님.🤓😎😆😲\n늘 열심히 하셔서 보기 좋아요.\n구독자 얘기가 매번 나오는데 너무 연연하지 않았음 하네요. 그냥 단단한 집을 지어가는 과정이라고 생각하심 좋지 않을까 싶어요. \n노력은 절대 배신하지 않아요. \n유노님 돼지고기 트라우

In [352]:
val=sentiment_predict(test_path,800,900)

['아니.. 양을 거의 두배로 늘리고 시간을 줄이는건.. 이건 도전메뉴가 아니고.. 유튜버들 놀리는거 같아서 화나네요..상해기님꺼 유노님꺼 히밥님꺼 영상 다 봤는데.. 솔직히 유노님 상해기님 못드실 정도면 일반인은 도전 엄두도 못낼 양..']
--> 50.25% 확률로 부정적인 댓글입니다.

['많이  먹는거 보다  맛있게  믝는 컨셉이 더 좋다 목소리도 좋고 말도 잘해서']
--> 59.29% 확률로 부정적인 댓글입니다.

['흥하시길!']
--> 59.28% 확률로 부정적인 댓글입니다.

['그냥 조용히 먹는 모습도 좋은데...... 너무']
--> 64.02% 확률로 부정적인 댓글입니다.

['노력하는 남자  유노  멋진남자 유노  힘내라~~힘 무조건  전진\n~~~~~']
--> 60.21% 확률로 부정적인 댓글입니다.

['🏆🥇최고예요  유노~👋🙆\u200d♀️']
--> 63.55% 확률로 부정적인 댓글입니다.

['진짜 잘먹는다 대리만족을']
--> 64.42% 확률로 부정적인 댓글입니다.

['아프리카tv 어제  마피아 떡볶이 보고  왔어요 구독  앞으로 홧팅입니다\n햄식이 같은  방송하시면 절대 안됨 갠방 전혀 안하고 모기짖만 하면 절대안됨']
-->  50.21% 확률로 긍정적인 댓글입니다.

['앗ㅡ일빠넹ㅡ응원해요 유노씨']
--> 63.21% 확률로 부정적인 댓글입니다.

['왤케 많이 먹어 윤호']
--> 64.42% 확률로 부정적인 댓글입니다.

['처음 보는데 목소리가 너무 좋아요']
--> 64.49% 확률로 부정적인 댓글입니다.

['새벽에 이 영상을보다니.... 떡볶이가 너무 먹고 싶어졌다....']
--> 63.21% 확률로 부정적인 댓글입니다.

['아구 맛 표현 명랑하네요^^ 꿀성대 꿀조합']
--> 62.23% 확률로 부정적인 댓글입니다.

['맛깔나게먹는다진짜']
--> 63.55% 확률로 부정적인 댓글입니다.

['그동안 돈내고 못먹은거 유노님이 대신 먹었다고 생각할게요~ 돈 안아까워~ ㅎㅎ']
--> 56.86% 확

['잘 봤습니다. 근데 주실 수 있을까요? 라는 말보다 주세요가 맞는 표현 일듯. 더더욱 번창하세요']
--> 52.60% 확률로 부정적인 댓글입니다.

['유노님도  얼렁 떡상해야델텐데']
--> 62.23% 확률로 부정적인 댓글입니다.

['착하고 겸손한 왕자님.🤓😎😆😲\n늘 열심히 하셔서 보기 좋아요.\n구독자 얘기가 매번 나오는데 너무 연연하지 않았음 하네요. 그냥 단단한 집을 지어가는 과정이라고 생각하심 좋지 않을까 싶어요. \n노력은 절대 배신하지 않아요. \n유노님 돼지고기 트라우마로 못먹는데 넘 대리만족되네요.😆😄😊🍎🍈🍉🍋\n더위 조심하시고 건강도 잘 챙기셔요.\n그리고 캡이에요.🍀🍀🍀👍']
--> 50.25% 확률로 부정적인 댓글입니다.

['🏆🥇최고예요 유노~👋🙆\u200d♀️']
--> 63.55% 확률로 부정적인 댓글입니다.

['유노님 덕분에 몇달만에 삼겹살을\n(눈으로) 먹었네요. 혹시 수령처가\n바뀌지 않았다면 만두 보내드릴테니\n집에서 드실 거 없을 때 드세요.']
-->  50.21% 확률로 긍정적인 댓글입니다.

['유노님이랑 밥먹음 나 없던 입맛이 살아날거 같아 최고!']
--> 59.29% 확률로 부정적인 댓글입니다.

['무한리필갈때 유노님 위장 빌릴수있으면 꿀이겠다...']
--> 62.23% 확률로 부정적인 댓글입니다.



In [369]:
stopwords_movie

['의',
 '가',
 '이',
 '은',
 '들',
 '는',
 '좀',
 '잘',
 '걍',
 '과',
 '도',
 '를',
 '으로',
 '자',
 '에',
 '와',
 '한',
 '하다']

In [363]:
lstm

,comment,sentiment_score
50,고릴라 느낌 난다 잘먹게 생깄~~🤣🤣,35.508558
51,하지마하지마 이런거하지마 ㅠㅠ\n\n내가말했잔아여 가만히있어도 얼굴이다했다고 ㅠㅠ굳...,44.028670
52,옛날부터 구독했는데 대단합니다,35.581738
53,맵찔이 유노님 한테는 지옥행 인데 ㅠㅠ\n넘넘고생해써요~ 물도 못머꾸 참느냐...,50.208956
54,기.. 기요미 ㅠㅅ ㅠ♡ 맵찔이 ㄹㅇㅋㅋ,37.771323
55,맛을 괜히 느껴 버렷어ㅋ\n인생살면서 4분44초가 이리 길엇던가\n느낌이 새로웟겟네...,48.780748
56,아 보면서 넘 웃었네요 ㅎㅎㅎ 죄송,35.978538
57,입술 퉁퉁 부은것봐 .. ㄷㄷ 무슨맛인지 궁금했는데 유체이탈 경험하는 맛이군요 ;;...,50.369990
58,저도맵찔이인데 보는것도매워서\n정신없이 물먹다가\n옷에물쏬았음ㅋㅋㅋ,40.706700
59,아우 절대 사먹지 말아야겠어요!! 진짜 속상해서 못보겠네 ㅠㅠ,38.801748


In [370]:
lstm=sentiment_predict(test_path,800,900)

['아니.. 양을 거의 두배로 늘리고 시간을 줄이는건.. 이건 도전메뉴가 아니고.. 유튜버들 놀리는거 같아서 화나네요..상해기님꺼 유노님꺼 히밥님꺼 영상 다 봤는데.. 솔직히 유노님 상해기님 못드실 정도면 일반인은 도전 엄두도 못낼 양..']
--> 50.25% 확률로 부정적인 댓글입니다.

['많이  먹는거 보다  맛있게  믝는 컨셉이 더 좋다 목소리도 좋고 말도 잘해서']
--> 59.29% 확률로 부정적인 댓글입니다.

['흥하시길!']
--> 59.28% 확률로 부정적인 댓글입니다.

['그냥 조용히 먹는 모습도 좋은데...... 너무']
--> 64.02% 확률로 부정적인 댓글입니다.

['노력하는 남자  유노  멋진남자 유노  힘내라~~힘 무조건  전진\n~~~~~']
--> 60.21% 확률로 부정적인 댓글입니다.

['🏆🥇최고예요  유노~👋🙆\u200d♀️']
--> 63.55% 확률로 부정적인 댓글입니다.

['진짜 잘먹는다 대리만족을']
--> 64.49% 확률로 부정적인 댓글입니다.

['아프리카tv 어제  마피아 떡볶이 보고  왔어요 구독  앞으로 홧팅입니다\n햄식이 같은  방송하시면 절대 안됨 갠방 전혀 안하고 모기짖만 하면 절대안됨']
-->  50.21% 확률로 긍정적인 댓글입니다.

['앗ㅡ일빠넹ㅡ응원해요 유노씨']
--> 63.21% 확률로 부정적인 댓글입니다.

['왤케 많이 먹어 윤호']
--> 64.42% 확률로 부정적인 댓글입니다.

['처음 보는데 목소리가 너무 좋아요']
--> 64.49% 확률로 부정적인 댓글입니다.

['새벽에 이 영상을보다니.... 떡볶이가 너무 먹고 싶어졌다....']
--> 62.23% 확률로 부정적인 댓글입니다.

['아구 맛 표현 명랑하네요^^ 꿀성대 꿀조합']
--> 62.23% 확률로 부정적인 댓글입니다.

['맛깔나게먹는다진짜']
--> 63.55% 확률로 부정적인 댓글입니다.

['그동안 돈내고 못먹은거 유노님이 대신 먹었다고 생각할게요~ 돈 안아까워~ ㅎㅎ']
--> 56.86% 확

['이 분 진짜 맛있게 많이 깔끔하게 잘 드시는데 ㅠㅠ 얼른 더더욱 뜨셨으면 좋겠습니다!유노님 화이팅~!!! -구독자 3천명때부터 보던 독자 올림.']
-->  50.21% 확률로 긍정적인 댓글입니다.

['잘 봤습니다. 근데 주실 수 있을까요? 라는 말보다 주세요가 맞는 표현 일듯. 더더욱 번창하세요']
--> 52.60% 확률로 부정적인 댓글입니다.

['유노님도  얼렁 떡상해야델텐데']
--> 62.23% 확률로 부정적인 댓글입니다.

['착하고 겸손한 왕자님.🤓😎😆😲\n늘 열심히 하셔서 보기 좋아요.\n구독자 얘기가 매번 나오는데 너무 연연하지 않았음 하네요. 그냥 단단한 집을 지어가는 과정이라고 생각하심 좋지 않을까 싶어요. \n노력은 절대 배신하지 않아요. \n유노님 돼지고기 트라우마로 못먹는데 넘 대리만족되네요.😆😄😊🍎🍈🍉🍋\n더위 조심하시고 건강도 잘 챙기셔요.\n그리고 캡이에요.🍀🍀🍀👍']
--> 50.25% 확률로 부정적인 댓글입니다.

['🏆🥇최고예요 유노~👋🙆\u200d♀️']
--> 63.55% 확률로 부정적인 댓글입니다.

['유노님 덕분에 몇달만에 삼겹살을\n(눈으로) 먹었네요. 혹시 수령처가\n바뀌지 않았다면 만두 보내드릴테니\n집에서 드실 거 없을 때 드세요.']
--> 50.54% 확률로 부정적인 댓글입니다.

['유노님이랑 밥먹음 나 없던 입맛이 살아날거 같아 최고!']
--> 59.29% 확률로 부정적인 댓글입니다.

['무한리필갈때 유노님 위장 빌릴수있으면 꿀이겠다...']
--> 62.23% 확률로 부정적인 댓글입니다.



In [338]:
test2

,channel,title,author,comment,comment_refined,like
0,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,유노-U_know\n\n\n\n \n 유노-U_know,이번 영상은 광고 아닙니다!! 챌린지 도전해서 경품 받고싶어서 영상 찍었어욥 ㅎㅎ..,이번 영상은 광고 아닙니다 챌린지 도전해서 경품 받고싶어서 영상 찍었어욥 ㅎㅎ,51
1,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,박재현,진짜 광고주들 찍으라고 보내는거 좆같다\n엄청유명항 사람도 아니고\n홍보는 하고싶고...,진짜 광고주들 찍으라고 보내는거 좆같다엄청유명항 사람도 아니고홍보는 하고싶고 돈은주...,19
2,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,만두,경품받고싶어서 찍엇다는게 넘나 귀야움 ㅠㅠㅠ,경품받고싶어서 찍엇다는게 넘나 귀야움 ㅠㅠㅠ,5
3,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,이수연,헉쓰 울 유노유노 입술이 순대가 되었네요...😱\n으악..내속이 다 쓰라리넹ㅜㅜㅜ\...,헉쓰 울 유노유노 입술이 순대가 되었네요으악내속이 다 쓰라리넹ㅜㅜㅜ속 괜찮으신가요ㅎ...,2
4,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,말뚝박자,11:37\n11:58\n12:09\n우유가 뭐 진화하낰ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n...,우유가 뭐 진화하낰ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ계속 커지넼ㅋㅋㅋㅋㅋㅋ,0
5,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,Baby Chanel tv,먹방 비제이 말고 그냥 남캠 비제이로 나가도 대성하실듯...! 말도 잘하시고 비쥬얼...,먹방 비제이 말고 그냥 남캠 비제이로 나가도 대성하실듯 말도 잘하시고 비쥬얼 ㅠㅠ ...,5
6,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,태주랑진기랑,아 웃으면 안되는뎅\n너무나 기엽다,아 웃으면 안되는뎅너무나 기엽다,13
7,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,김다연,마음이 아풉니다.. ..,마음이 아풉니다,8
8,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,monaco9593,유노님!!\n매운건 이제 도전하지마요 ㅠㅠ\n제속이 다 매운듯 하네!!\n그래도 멋...,유노님매운건 이제 도전하지마요 ㅠㅠ제속이 다 매운듯 하네그래도 멋지십니다 늘 응원...,19
9,유노-U_know,한국에서 가장 맵다는 불마왕라면 챌린지 도전해봤습니다! 1분 44초 안에 불마왕 볶...,잠이 안와,아 진짜 광고주들 이런거 안 줬으면 좋겟어ㅜㅜㅜㅜ 맛있으려고 음식 먹지 아프려고 먹...,아 진짜 광고주들 이런거 안 줬으면 좋겟어ㅜㅜㅜㅜ 맛있으려고 음식 먹지 아프려고 먹...,2


In [334]:
test2=pd.read_csv('유노-U_know_meta_comment.csv')